In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

<h1>recomendacion_usuario</h1>

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [3]:
# Cargamos el archivo CSV a utilizar:
games_df = pd.read_csv(r"..\Archivos Producidos\games.csv")

games_df.dtypes

app_name                     object
release_year                  int64
item_id                       int64
developer                    object
Action                        int64
Adventure                     int64
Animation &amp; Modeling      int64
Audio Production              int64
Casual                        int64
Design &amp; Illustration     int64
Early Access                  int64
Education                     int64
Free to Play                  int64
Indie                         int64
Massively Multiplayer         int64
Photo Editing                 int64
RPG                           int64
Racing                        int64
Simulation                    int64
Software Training             int64
Sports                        int64
Strategy                      int64
Utilities                     int64
Video Production              int64
Web Publishing                int64
dtype: object

In [4]:
# Seleccionamos las columnas que nos serán de utilidad:
df_recom_game = games_df[['item_id', 'Action',
       'Adventure', 'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing']]

df_recom_game.head()

,item_id,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,761140,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,643980,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
2,670290,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
3,767400,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,772540,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
# Cargamos la lista de usuarios:
user_df = pd.read_csv(r"..\Archivos Producidos\users.csv")[["user_id", "item_id"]]

user_df.head()

,user_id,item_id
0,76561197970982479,10
1,Js41637,10
2,Evcentric,1200
3,Riot-punch,10
4,Doctr,300


In [6]:
# Cargamos la lista de nombres de los videojuegos:
videogames_df = pd.read_csv(r"..\Archivos Producidos\games.csv")[["app_name", "item_id"]]

videogames_df.head()

,app_name,item_id
0,Lost summoner kitty,761140
1,Ironbound,643980
2,Real pool 3d - poolians,670290
3,弹炸人2222,767400
4,Battle royale trainer,772540


Una vez ya filtrado del DataFrame, creamos el modelo de recomendación:

In [7]:
# Utilizaremos nuevamente el modelo creado en "recomendacion_juego.ipynb":

def recomendacion_juego(item_id):

    # Obtenemos la fila correspondiente al juego dado por "item_id":
    juego = df_recom_game[df_recom_game["item_id"] == item_id].iloc[:, 1:]

    # Excluímos el juego del DataFrame antes de calcular la similitud (evitamos que el modelo nos recomiende el mismo juego):
    df_sin_juego = df_recom_game[df_recom_game["item_id"] != item_id]

    # Calculamos la similitud de coseno entre el juego y todos los demás juegos:
    similitudes = cosine_similarity(juego, df_sin_juego.iloc[:, 1:])

    # Convertimos las similitudes en un DataFrame:
    similitudes_df = pd.DataFrame(similitudes.flatten(), columns=["similitud"])

    # Encontramos los 5 juegos más similares:
    juegos_similares = df_sin_juego.loc[similitudes_df.sort_values(by="similitud", ascending=False).index[:5], "item_id"]

    # Devolvemos el resultado final:
    return list(juegos_similares)

In [8]:
# Creamos el modelo de recomendación:
# Este modelo de recomendación utilizará el modelo anterior "recomendacion_juego":
def recomendacion_usuario(usuario):

    # Capitalizamos la variable usuario:
    usuario = usuario.strip().capitalize()


    # Obtenemos el "item_id" del juego que ha jugado el usuario:
    item_id = user_df[user_df["user_id"] == usuario]["item_id"].values[0]

    # Llamamos a la función anteriormente creada y creamos una lista con los nombres de los videojuegos en lugar de sus IDs:
    lista_ids = recomendacion_juego(item_id)
    lista_nombres = []
    for i in lista_ids:
        game_name = videogames_df[videogames_df["item_id"] == i]["app_name"].values[0]
        lista_nombres.append(game_name)

    # Devolvemos el resultado final:
    return lista_nombres

In [9]:
# Probamos el modelo:
print(recomendacion_usuario("Fr0stedline"))
print(recomendacion_usuario("76561198315899135"))
print(recomendacion_usuario("  arKPLayS7 "))
print(recomendacion_usuario("Arkplays7"))
print(recomendacion_usuario("doctr"))

['The bureau: xcom declassified - light plasma pistol', 'Eve: valkyrie – warzone drifter pack', 'Eve: valkyrie – warzone x50 gold capsule', 'Sacred citadel', 'Aliens: colonial marines - movie map pack']
['Never alone (kisima ingitchuna)', 'Grass max', 'Osteya', "Glorch's great escape: walking is for chumps", 'Arrow heads - soundtrack']
['Apb reloaded urban survival pack', 'Warface - jungle starter pack', 'Tanki x: mercenary flamethrower', 'Tanki x: antaeus skirmisher', 'Piercing blow']
['Apb reloaded urban survival pack', 'Warface - jungle starter pack', 'Tanki x: mercenary flamethrower', 'Tanki x: antaeus skirmisher', 'Piercing blow']
['The bureau: xcom declassified - light plasma pistol', 'Eve: valkyrie – warzone drifter pack', 'Eve: valkyrie – warzone x50 gold capsule', 'Sacred citadel', 'Aliens: colonial marines - movie map pack']


Guardamos el DataFrame utilizado:

In [11]:
# Guardamos el dataframe "user_df":
user_df.to_csv(r"..\Archivos Producidos\Data para Endpoints\user_df.csv", index=False)

In [12]:
# Guardamos el dataframe "videogames_df":
videogames_df.to_csv(r"..\Archivos Producidos\Data para Endpoints\videogames_df.csv", index=False)